# 01_basic_extraction.ipynb
## 基本的なPDF抽出機能のテスト

このノートブックでは、PDFからテキスト情報を抽出する基本機能をテストします。
1. PyPDF2によるテキスト抽出
2. PDF画像としての処理（画像ベースPDF用）
3. 簡易OCR機能テスト
4. 抽出結果の確認と評価

In [30]:
# 必要なライブラリのインポート
import os
import sys
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# PyPDF2を使用したPDF処理
import PyPDF2

# 画像処理用
from PIL import Image
import pdf2image

# OCR用
import pytesseract

# モジュールの親ディレクトリをパスに追加
sys.path.append('..')

# プロジェクトルートパスの設定
ROOT_DIR = Path('..').resolve()
DATA_DIR = ROOT_DIR / 'data'
INPUT_DIR = DATA_DIR / 'input'
OUTPUT_DIR = DATA_DIR / 'output'
TEMP_DIR = DATA_DIR / 'temp'

# 出力ディレクトリを作成
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(TEMP_DIR, exist_ok=True)

plt.rcParams['font.family'] = 'Noto Sans CJK JP'

## 1. PDFファイルの読み込みと基本情報の確認

In [31]:
# 利用可能なPDFファイルの一覧を取得
pdf_files = list(INPUT_DIR.glob('*.pdf'))
print(f"利用可能なPDFファイル: {[p.name for p in pdf_files]}")

# テスト対象のPDFファイルを選択
test_pdf = pdf_files[0]  # インデックスを変更して別のファイルを選択可能
print(f"選択したPDFファイル: {test_pdf.name}")

利用可能なPDFファイル: ['2023年度活動計算書.pdf']
選択したPDFファイル: 2023年度活動計算書.pdf


In [20]:
# PDFの基本情報を取得
def get_pdf_info(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        info = {
            'ページ数': len(reader.pages),
            'メタデータ': reader.metadata,
            'ファイルサイズ': os.path.getsize(pdf_path) / 1024,  # KB単位
        }
        return info

pdf_info = get_pdf_info(test_pdf)
print("PDF基本情報:")
for key, value in pdf_info.items():
    print(f"{key}: {value}")

PDF基本情報:
ページ数: 2
メタデータ: {'/Author': '', '/CreationDate': "D:20240708135349+09'00'", '/ModDate': "D:20240708135349+09'00'", '/Producer': 'Microsoft: Print To PDF', '/Title': b'z\x97\xf8(y\x9a^\xb6);\xd5\xd5\xba\x8e\x9e\x97\x9d\x80\x98p\x88r\x8a\x9ep}\x9e ,23\x1f)_\xcf\x1a\x7f\x8d.xlsx'}
ファイルサイズ: 100.90625


## 2. PyPDF2によるテキスト抽出

In [22]:
# PyPDF2を使用したテキスト抽出
def extract_text_with_pypdf2(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text_by_page = []
        
        for page_num, page in enumerate(reader.pages):
            text = page.extract_text()
            text_by_page.append({
                'page_num': page_num + 1,
                'text': text
            })
            
        return text_by_page

pypdf2_text = extract_text_with_pypdf2(test_pdf)

# 抽出結果の表示
# 全ページのテキストを結合して表示
all_text = '\n\n'.join([page['text'] for page in pypdf2_text])
print("=== 全テキスト ===\n")
print(all_text)

=== 全テキスト ===

【経常収益】
  【受取会費】
    受取入会金 6,000 
    正会員受取会費 311,000 
    賛助会員受取会費 47,000 
    利用会員受取会費 42,000 406,000 
  【受取寄付金】
    受取寄付金 5,056,942 
  【受取助成金等】
    受取助成金 130,000 
  【事業収益】
    事業　収益 2,347,170 
    受託事業収益（公共部門） 25,643,785 
    受託事業収益（民間部門） 1,624,712 29,615,667 
  【その他収益】
    受取　利息 204 
    雑　収　益 1,200 1,404 
        経常収益  計 35,210,013 
【経常費用】
  【事業費】
    （人件費）
      給料　手当 9,810,255 
      法定福利費 990,563 
      退職給付費用(事) 50,983 
      通　勤　費 345,503 
      福利厚生費 87,822 
        人件費計 11,285,126 
    （その他経費）
      売上　原価 277,200 
      業務委託費 4,223,086 
      諸　謝　金 9,697,259 
      印刷製本費 198,135 
      会　議　費 2,800 
      旅費交通費 367,127 
      通信運搬費 783,100 
      消耗品　費 327,050 
      水道光熱費 180,209 
      賃　借　料 1,625,650 
      保　険　料 27,284 
      諸　会　費 138,000 
      租税　公課 1,359,770 
      支払手数料 267,641 
      支払寄付金 3,000,000 
      修　繕　費 20,229 
      新聞図書費 113,126 
      慶　弔　費 9,618 
      雑　　　費 100,196 
        その他経費計 22,717,480 
          事業費  計 34,002,606 
  【管理費】
    （人件費

In [35]:
import os
import re
import pandas as pd
import PyPDF2
from pathlib import Path

# モジュールの親ディレクトリをパスに追加
sys.path.append('..')

# プロジェクトルートパスの設定
ROOT_DIR = Path('..').resolve()
DATA_DIR = ROOT_DIR / 'data'
INPUT_DIR = DATA_DIR / 'input'
OUTPUT_DIR = DATA_DIR / 'output'
TEMP_DIR = DATA_DIR / 'temp'

# 出力ディレクトリを作成
os.makedirs(OUTPUT_DIR, exist_ok=True)

# PDFファイルのパスを指定（あるいはINPUT_DIRから取得）
pdf_path = INPUT_DIR / '2023年度活動計算書.pdf'

# PyPDF2を使用したテキスト抽出
def extract_text_with_pypdf2(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text_by_page = []
        
        for page_num, page in enumerate(reader.pages):
            text = page.extract_text()
            text_by_page.append({
                'page_num': page_num + 1,
                'text': text
            })
            
        return text_by_page

# 勘定科目と金額を抽出する関数
def extract_account_items(text):
    # データを格納するためのリスト
    accounts = []
    
    # 行ごとに処理
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # 勘定科目と金額のパターンにマッチするか確認
        # パターン: [科目名] [金額（数字とカンマ）] [右端の合計]
        pattern = r'(.+?)\s+(\d{1,3}(?:,\d{3})*|\d+)(?:\s+(\d{1,3}(?:,\d{3})*|\d+))?$'
        match = re.search(pattern, line)
        
        if match:
            account_name = match.group(1).strip()
            amount = match.group(2).strip().replace(',', '')
            total_amount = match.group(3).replace(',', '') if match.group(3) else None
            
            # 特殊文字（全角スペースなど）を置換
            account_name = account_name.replace('　', ' ').strip()
            
            # 「【」「】」で囲まれた項目は大分類として扱う
            if account_name.startswith('【') and account_name.endswith('】'):
                category = account_name.strip('【】')
                accounts.append({
                    'category': category,
                    'subcategory': None,
                    'account_name': category,
                    'amount': amount,
                    'total_amount': total_amount
                })
            else:
                # 直前の大分類を取得
                current_category = accounts[-1]['category'] if accounts and 'category' in accounts[-1] else None
                
                accounts.append({
                    'category': current_category,
                    'subcategory': None,  # 必要に応じて設定
                    'account_name': account_name,
                    'amount': int(amount),
                    'total_amount': int(total_amount) if total_amount else None
                })
    
    return accounts

# 勘定科目の階層構造を整理
def organize_structure(accounts, structure):
    # 階層構造をもとに勘定科目を整理
    organized = {}
    
    # 現在の大分類・中分類を追跡
    current_category = None
    current_subcategory = None
    
    for item in accounts:
        account_name = item['account_name']
        amount = item['amount']
        
        # 大分類が変わった場合
        if account_name in structure.keys():
            current_category = account_name
            current_subcategory = None
            if current_category not in organized:
                organized[current_category] = {'items': {}, 'total': None}
        
        # 中分類が変わった場合
        elif current_category and account_name in structure.get(current_category, {}):
            current_subcategory = account_name
            if current_subcategory not in organized[current_category]['items']:
                organized[current_category]['items'][current_subcategory] = {'items': {}, 'total': None}
        
        # 明細項目の場合
        elif current_category and current_subcategory:
            organized[current_category]['items'][current_subcategory]['items'][account_name] = amount
            
            # 合計項目の場合は中分類の合計としても記録
            if '合計' in account_name or '計' in account_name:
                organized[current_category]['items'][current_subcategory]['total'] = amount
        
        # その他のケース（大分類直下の項目など）
        elif current_category:
            organized[current_category]['items'][account_name] = amount
            
            # 合計項目の場合は大分類の合計としても記録
            if '合計' in account_name or '計' in account_name:
                organized[current_category]['total'] = amount
    
    return organized

# より詳細な処理用の拡張関数
def extract_detailed_accounts(text):
    """
    より詳細な勘定科目抽出と階層構造の認識を行う関数
    活動計算書の特徴的なフォーマットを考慮
    """
    accounts = []
    lines = text.split('\n')
    
    current_category = None
    current_subcategory = None
    current_sub_subcategory = None
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        # 「活動計算書」や「特定非営利活動法人」などのヘッダー行は無視
        if '活動計算書' in line or '特定非営利活動法人' in line or '[税込]' in line or ('自' in line and '至' in line):
            continue
            
        # 大分類 (【経常収益】など)
        if re.match(r'【(.+)】', line):
            current_category = re.match(r'【(.+)】', line).group(1)
            current_subcategory = None
            current_sub_subcategory = None
            
            accounts.append({
                'category': current_category,
                'subcategory': None,
                'sub_subcategory': None,
                'account_name': current_category,
                'amount': None,
                'total_amount': None,
                'level': 1,
                'is_header': True
            })
            continue
            
        # 中分類 (【受取会費】など) - 大分類の下位階層
        if re.match(r'\s*【(.+)】', line):
            current_subcategory = re.match(r'\s*【(.+)】', line).group(1)
            current_sub_subcategory = None
            
            accounts.append({
                'category': current_category,
                'subcategory': current_subcategory,
                'sub_subcategory': None,
                'account_name': current_subcategory,
                'amount': None,
                'total_amount': None,
                'level': 2,
                'is_header': True
            })
            continue
            
        # さらに小分類（人件費）など
        if re.match(r'\s*（(.+)）', line):
            current_sub_subcategory = re.match(r'\s*（(.+)）', line).group(1)
            
            accounts.append({
                'category': current_category,
                'subcategory': current_subcategory,
                'sub_subcategory': current_sub_subcategory,
                'account_name': current_sub_subcategory,
                'amount': None,
                'total_amount': None,
                'level': 3,
                'is_header': True
            })
            continue
            
        # 勘定科目と金額
        pattern = r'(.+?)\s+(\d{1,3}(?:,\d{3})*|\d+)(?:\s+(\d{1,3}(?:,\d{3})*|\d+))?$'
        match = re.search(pattern, line)
        
        if match:
            account_name = match.group(1).strip()
            amount_str = match.group(2).replace(',', '')
            total_amount_str = match.group(3).replace(',', '') if match.group(3) else None
            
            # 金額を整数に変換
            try:
                amount = int(amount_str)
            except ValueError:
                amount = None
                
            try:
                total_amount = int(total_amount_str) if total_amount_str else None
            except ValueError:
                total_amount = None
            
            # 科目名に含まれる全角スペースを半角に変換
            account_name = account_name.replace('　', ' ').strip()
            
            # 計や合計が付く項目はサブカテゴリの合計項目と判断
            is_subtotal = '計' in account_name
            
            # 階層レベルを判断
            if current_sub_subcategory:
                level = 4
            elif current_subcategory:
                level = 3
            else:
                level = 2
            
            accounts.append({
                'category': current_category,
                'subcategory': current_subcategory,
                'sub_subcategory': current_sub_subcategory,
                'account_name': account_name,
                'amount': amount,
                'total_amount': total_amount,
                'level': level,
                'is_header': False,
                'is_subtotal': is_subtotal
            })
    
    return accounts

# 勘定科目構造を定義
account_structure = {
    '経常収益': {
        '受取会費': ['受取入会金', '正会員受取会費', '賛助会員受取会費', '利用会員受取会費', '受取会費合計'],
        '受取寄付金': ['受取寄付金'],
        '受取助成金等': ['受取助成金'],
        '事業収益': ['事業収益', '受託事業収益（公共部門）', '受託事業収益（民間部門）', '事業収益合計'],
        'その他収益': ['受取利息', '雑収益', 'その他収益合計'],
        '経常収益合計': []
    },
    '経常費用': {
        '事業費': {
            '人件費': ['給料手当', '法定福利費', '退職給付費用(事)', '通勤費', '福利厚生費', '人件費計'],
            'その他経費': [
                '売上原価', '業務委託費', '諸謝金', '印刷製本費', '会議費', '旅費交通費', 
                '通信運搬費', '消耗品費', '水道光熱費', '賃借料', '保険料', '諸会費', 
                '租税公課', '支払手数料', '支払寄付金', '修繕費', '新聞図書費', '慶弔費', 
                '雑費', 'その他経費計'
            ],
            '事業費計': []
        },
        '管理費': {
            '人件費': ['給料手当', '法定福利費', '退職給付費用', '通勤費', '福利厚生費', '人件費計'],
            'その他経費': [
                '諸謝金', '印刷製本費', '旅費交通費', '通信運搬費', '消耗品費', 
                '新聞図書費', '修繕費', '水道光熱費', '賃借料', '保険料', '慶弔費', 
                '支払手数料', '雑費', 'その他経費計'
            ],
            '管理費計': []
        },
        '経常費用計': []
    },
    '当期経常増減額': [],
    '経常外収益': ['過年度損益修正益', '経常外収益計'],
    '経常外費用': ['経常外費用計'],
    '税引前当期正味財産増減額': [],
    '法人税、住民税及び事業税': [],
    '当期正味財産増減額': [],
    '前期繰越正味財産額': [],
    '次期繰越正味財産額': []
}

def main():
    # テキスト抽出を実行
    pdf_text = extract_text_with_pypdf2(pdf_path)
    
    # 全ページのテキストを結合
    all_text = '\n\n'.join([page['text'] for page in pdf_text])
    
    # 簡易抽出
    accounts = extract_account_items(all_text)
    df = pd.DataFrame(accounts)
    
    print("抽出された勘定科目:")
    print(df.head())
    
    # 詳細な抽出
    detailed_accounts = extract_detailed_accounts(all_text)
    detailed_df = pd.DataFrame(detailed_accounts)
    
    # 結果を保存
    df.to_csv(OUTPUT_DIR / 'accounts_extracted.csv', index=False, encoding='utf-8-sig')
    detailed_df.to_csv(OUTPUT_DIR / 'detailed_accounts.csv', index=False, encoding='utf-8-sig')
    
    print("勘定科目抽出が完了しました。")
    print(f"結果は {OUTPUT_DIR} に保存されました。")

if __name__ == "__main__":
    main()

抽出された勘定科目:
  category subcategory account_name   amount  total_amount
0     None        None        受取入会金     6000           NaN
1     None        None      正会員受取会費   311000           NaN
2     None        None     賛助会員受取会費    47000           NaN
3     None        None     利用会員受取会費    42000      406000.0
4     None        None        受取寄付金  5056942           NaN
勘定科目抽出が完了しました。
結果は /home/alma/pdf_extractor/data/output に保存されました。


## 3. PyPDF2での抽出が不十分な場合は画像ベースの処理を実施

In [ ]:
# 抽出テキストの品質評価（簡易）
def evaluate_text_quality(text):
    # テキストが空か短すぎる場合は低品質と判断
    if not text or len(text.strip()) < 50:
        return False
    
    # 特定のキーワードが含まれているか確認（活動計算書の場合）
    keywords = ['活動', '計算', '収入', '支出', '合計']
    keyword_count = sum(1 for kw in keywords if kw in text)
    
    # キーワードが2つ以上含まれていれば十分と判断
    return keyword_count >= 2

# PyPDF2で抽出したテキストの品質を評価
text_quality = [evaluate_text_quality(page['text']) for page in pypdf2_text]
print(f"ページごとのテキスト品質: {text_quality}")

## 5. テキストからの活動計算書のデータ抽出（簡易版）

In [34]:
# 活動計算書の主要項目に関する簡易パターンマッチング
def extract_financial_items(text):
    import re
    
    items = {}
    
    # 収入・支出・合計などの基本パターン
    patterns = {
        '経常収益': r'経常収益[\s\S]*?合計[\s]*([\d,]+)',
        '経常費用': r'経常費用[\s\S]*?合計[\s]*([\d,]+)',
        '当期経常増減額': r'当期経常増減額[\s]*([\d,▲-]+)',
        '当期正味財産増減額': r'当期正味財産増減額[\s]*([\d,▲-]+)',
        '正味財産期首残高': r'正味財産期首残高[\s]*([\d,]+)',
        '正味財産期末残高': r'正味財産期末残高[\s]*([\d,]+)'
    }
    
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            # カンマを除去して数値化
            value_str = match.group(1).replace(',', '')
            # マイナス記号の処理（▲や-）
            if '▲' in value_str or '-' in value_str:
                value_str = value_str.replace('▲', '').replace('-', '')
                items[key] = -int(value_str)
            else:
                items[key] = int(value_str)
                
    return items

# 抽出テキストから財務項目を取得
# すべてのページのテキストを結合
all_text = '\n'.join([page['text'] for page in pypdf2_text])

# OCR結果も追加
if ocr_results:
    all_text += '\n' + '\n'.join([page['text'] for page in ocr_results])

# 財務項目の抽出
financial_items = extract_financial_items(all_text)

# 結果の表示
print("抽出された財務項目:")
for key, value in financial_items.items():
    print(f"{key}: {value:,}円")

抽出された財務項目:
当期経常増減額: 156,875円
当期正味財産増減額: 447,538円


## 6. 抽出結果の保存

In [ ]:
# 結果をJSONで保存
result = {
    'pdf_info': pdf_info,
    'extraction_method': 'PyPDF2',
    'ocr_used': len(ocr_results) > 0,
    'financial_items': financial_items,
    'text_quality': {f"page_{i+1}": quality for i, quality in enumerate(text_quality)}
}

output_file = OUTPUT_DIR / f"{test_pdf.stem}_extraction_result.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"結果を保存しました: {output_file}")

## 7. 抽出結果の評価

In [ ]:
# 抽出結果の評価
def evaluate_extraction(financial_items):
    # 主要な項目が抽出できたか確認
    key_items = ['経常収益', '経常費用', '当期経常増減額', '正味財産期末残高']
    extracted_keys = financial_items.keys()
    
    found_items = [item for item in key_items if item in extracted_keys]
    missing_items = [item for item in key_items if item not in extracted_keys]
    
    # 整合性チェック（例: 収益-費用=増減額）
    consistency_errors = []
    if all(k in financial_items for k in ['経常収益', '経常費用', '当期経常増減額']):
        expected_diff = financial_items['経常収益'] - financial_items['経常費用']
        actual_diff = financial_items['当期経常増減額']
        if expected_diff != actual_diff:
            consistency_errors.append(
                f"経常収益-経常費用 ({expected_diff:,}円) ≠ 当期経常増減額 ({actual_diff:,}円)"
            )
    
    # 評価結果
    evaluation = {
        '抽出率': len(found_items) / len(key_items) if key_items else 0,
        '抽出できた項目': found_items,
        '抽出できなかった項目': missing_items,
        '整合性エラー': consistency_errors
    }
    
    return evaluation

# 抽出結果の評価を実施
evaluation = evaluate_extraction(financial_items)

print(f"抽出率: {evaluation['抽出率']*100:.1f}%")
print(f"抽出できた項目: {', '.join(evaluation['抽出できた項目'])}")

if evaluation['抽出できなかった項目']:
    print(f"抽出できなかった項目: {', '.join(evaluation['抽出できなかった項目'])}")
else:
    print("すべての主要項目が抽出できました。")
    
if evaluation['整合性エラー']:
    print("整合性エラー:")
    for error in evaluation['整合性エラー']:
        print(f"- {error}")
else:
    print("整合性チェックは問題ありませんでした。")

## 8. まとめと次のステップ

### 抽出テスト結果のまとめ

- PyPDF2によるテキスト抽出の品質: ページごとに異なる
- OCRの必要性: テキスト品質が低いページに対して実施
- 財務項目の抽出精度: 主要項目の抽出率で評価
- 整合性確認: 項目間の関係性が正しいか検証

### 次のステップ

1. **OCR品質の向上**
   - 画像前処理の改善 (コントラスト調整、ノイズ除去等)
   - OCRエンジンのパラメータ最適化

2. **パターン認識の強化**
   - より複雑な財務項目の抽出パターン開発
   - 階層構造の認識改善

3. **複数形式への対応**
   - 異なる様式の活動計算書への対応
   - 形式に応じた抽出戦略の切り替え

4. **AI活用の検討**
   - 抽出したテキストからのAIによる構造化支援
   - 整合性チェックの高度化

次のノートブック `02_ocr_testing.ipynb` ではOCR機能に特化したテストを行います。